In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import re
import os
import warnings

import pandas as pd
import numpy as np
import pandasql as psql

from solution import *

In [ ]:
import re
import os
import warnings

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from autokeras import StructuredDataRegressor

In [24]:
pd.set_option("display.max_rows", None, "display.max_columns", None) # Let pandas print the entire dataframe
warnings.filterwarnings('ignore')
%matplotlib inline

In [25]:
# input_file = 
# output_file =  

In [93]:
input_file= 'data\device_activations.csv'
data = pd.read_csv(input_file)
data['time'] = data['time'].apply(lambda dt: dt[:-5] + '00:00')
query = """
select 
    device, 
    time, 
    sum(device_activated) device_activated 
from data
    group by device,time
"""
data = psql.sqldf(query)
data['time'] = pd.to_datetime(data['time'])

new_data = data.set_index(
    ['time', 'device']
).unstack(
    fill_value=0
).asfreq(
    'H', fill_value=0
).stack().sort_index(level=1).reset_index()
#set(new_data.device_activated.values == data.device_activated.values)

new_data = extract_date_info(new_data)

In [194]:
query = """
select df_merge.device,
df_merge.day_name,
df_merge.hour,
data.device_activated
from df_merge
left join data 
on df_merge.device = data.device
and df_merge.day_name = data.day_name
and df_merge.hour = data.hour
"""
psql.sqldf(query)

,device,day_name,hour,device_activated
0,device1,Friday,4,None
1,device1,Friday,5,None
2,device1,Friday,6,None
3,device1,Friday,7,None
4,device1,Friday,8,None
5,device1,Friday,9,None
6,device1,Friday,10,None
7,device1,Friday,11,None
8,device1,Friday,12,None
9,device1,Friday,13,None


In [55]:
query = """
select day_name,count(*) num_day_name
from data
group by day_name
"""
psql.sqldf(query)


,day_name,num_day_name
0,Friday,1443
1,Monday,1634
2,Saturday,1
3,Sunday,20
4,Thursday,1974
5,Tuesday,1914
6,Wednesday,2059


In [58]:
df_merge.head()

,device,time
0,device1,2016-07-01 00:00:00
1,device1,2016-07-01 01:00:00
2,device1,2016-07-01 02:00:00
3,device1,2016-07-01 03:00:00
4,device1,2016-07-01 04:00:00


In [8]:
query = """
select device,count(*) num_device
from data
group by device
order by num_device desc

"""
psql.sqldf(query)

,device,num_device
0,device_2,2930
1,device_6,2134
2,device_5,1256
3,device_1,922
4,device_4,901
5,device_3,741
6,device_7,161


In [9]:
query = """
with week_device_capacity as (
select week, device, count(*) week_device_capacity
from data 
group by week,device
),

week_total_capacity as (
select week,count(*) week_total_capacity 
from data 
group by week),

table_join as(
select
*
from week_device_capacity
left join week_total_capacity
using(week))

select week,device,week_device_capacity,week_total_capacity,week_device_capacity*1.0/week_total_capacity*1. as ratio
from table_join
order by week, ratio desc

"""
psql.sqldf(query)

,week,device,week_device_capacity,week_total_capacity,ratio
0,26,device_2,69,117,0.589744
1,26,device_3,26,117,0.222222
2,26,device_6,12,117,0.102564
3,26,device_4,7,117,0.059829
4,26,device_5,3,117,0.025641
5,27,device_2,355,1172,0.302901
6,27,device_6,251,1172,0.214164
7,27,device_5,175,1172,0.149317
8,27,device_4,168,1172,0.143345
9,27,device_3,129,1172,0.110068


In [10]:
query = """
select week,day,hour,device,count(*) num_record
from data
group by week,day,hour,device
order by time,device

"""
psql.sqldf(query)

,week,day,hour,device,num_record
0,26,1,4,device_6,1
1,26,1,6,device_2,3
2,26,1,7,device_6,6
3,26,1,7,device_2,12
4,26,1,7,device_4,4
5,26,1,8,device_2,6
6,26,1,8,device_6,5
7,26,1,8,device_4,3
8,26,1,8,device_3,2
9,26,1,8,device_5,3


In [11]:
query = """
with get_record as (
select week,day,hour,device,count(*) num_record
from data
group by week,day,hour,device
),

final as(
select day,device,hour, 
sum(num_record) OVER(PARTITION BY device,day,hour) num_record_device_hour,
sum(num_record) OVER(PARTITION BY device,day) num_record_device_day
from get_record
)

select day,device, hour,num_record_device_hour*1./num_record_device_day ratio
from final

"""
data_processed = psql.sqldf(query)

In [12]:
data_processed

,day,device,hour,ratio
0,1,device_1,11,0.062500
1,1,device_1,14,0.312500
2,1,device_1,15,0.312500
3,1,device_1,16,0.250000
4,1,device_1,17,0.062500
5,2,device_1,7,0.085714
6,2,device_1,8,0.085714
7,2,device_1,9,0.171429
8,2,device_1,10,0.085714
9,2,device_1,11,0.085714
